# **Create datasets from bSSFP at 0.55T**
#### DATA
The CINE data was acquired for different slices in a 0.55T scanner

## **libraries**

In [1]:
# basic libraries
import numpy as onp
import jax.numpy as np 
import matplotlib.pyplot as plt  
import os
# bart
from inrmri.bart import bart_acquisition_from_arrays

In [2]:
from inrmri.utils_rdls import preprocess_data, get_csm_format
from inrmri.utils_rdls import plot_angles_and_times_from_xdata, balance_spokes_duplicating
from inrmri.utils_rdls import plot_traj, plot_kdata, plot_csm, plot_zoom_images

In [3]:
from inrmri.utils_rdls import load_acq_data, get_inf_about_data, filtering_spokes, plot_spokes_and_hb, sort_data_by_bin, get_csm_format, preprocess_data
from inrmri.utils_rdls import get_exp_weight_descrite, plot_angles_and_times_from_xdata, balance_spokes_duplicating

In [4]:
from inrmri.utils_rdls import create_folder, save_frames_as_gif_with_pillow, recon_bart

## **1. - Base data**

In [5]:
# input folders
volunteer_name      = 'FB' # 'MP', 'FH', 'TC', 'RF', 'JB', 'FE', 'DB', 'FP', 'AA'
slice_num_list      = [1, 2, 3, 4, 5, 6, 7, 8]
nslices             = 8
eliminated_spokes   = 20
# parameters
nbins                 = 30
# computation folders
bart_files_folder     = '/mnt/workspace/datasets/pulseqCINE/bart_files/'
# output folder
dataset_name          = 'DATA_0.55T'
base_folder           = '/mnt/workspace/datasets/pulseqCINE/' + dataset_name + '/' + volunteer_name + '/'
train_data_folder     = base_folder + '/traindata/'
recons_folder         = base_folder + '/recons/' 
raw_data_folder       = base_folder + '/datasets/'

In [6]:
# reset folder
create_folder(bart_files_folder, reset=True)
# create folder
create_folder(train_data_folder, reset=False)
create_folder(recons_folder, reset=False)

## Loop

In [7]:
for slice_num in slice_num_list:
    # paths
    dataset_name        = 'slice_' + str(slice_num) +'_' + str(nslices) +'_nbins' + str(nbins)
    dataset_name_gt     = 'slice_' + str(slice_num) +'_' + str(nslices) +'_nbins' + str(nbins) + '_gt'
    dataset_recon_folder      = recons_folder + dataset_name + '/'
    path_save                 = train_data_folder + dataset_name + '.npz'
    path2folder = raw_data_folder + dataset_name + '.mat'
    path2folder_gt = raw_data_folder + dataset_name_gt + '.mat'
    # load data
    bins, cbins, kdata, ktraj = load_acq_data(path2folder)
    n_spokes, n_readout, n_coils, n_bins, n_dims = get_inf_about_data(ktraj, kdata, cbins)
    bins_gt, cbins_gt, kdata_gt, ktraj_gt = load_acq_data(path2folder_gt)
    n_spokes_gt, n_readout_gt, n_coils_gt, n_bins_gt, n_dims_gt = get_inf_about_data(ktraj_gt, kdata_gt, cbins_gt)
    # --- Preprocess data  ---
    bins_filtered, kdata_filtered, ktraj_filtered, hb_filtered = filtering_spokes(eliminated_spokes, bins, kdata, ktraj)
    bins_filtered_gt, kdata_filtered_gt, ktraj_filtered_gt, hb_filtered_gt = filtering_spokes(eliminated_spokes, bins_gt, kdata_gt, ktraj_gt)
    # sort data
    kdata_sorted, ktraj_sorted, bins_sorted, hb_sorted = sort_data_by_bin(bins_filtered, kdata_filtered, ktraj_filtered, hb_filtered)
    kdata_sorted_gt, ktraj_sorted_gt, bins_sorted_gt, hb_sorted_gt = sort_data_by_bin(bins_filtered_gt, kdata_filtered_gt, ktraj_filtered_gt, hb_filtered_gt)
    # times
    times = onp.array(bins_sorted)/n_bins 
    times_gt = onp.array(bins_sorted_gt)/n_bins_gt 
    # --- D-DIP dataset ---
    Y_data, X_data, spclim = preprocess_data(times, kdata_sorted, ktraj_sorted)
    Y_data_gt, X_data_gt, spclim_gt = preprocess_data(times_gt, kdata_sorted_gt, ktraj_sorted_gt)
    # --- BART: COILS + RECONS ---
    csm, hollow_mask, recon_grasp, recon_sense, time_grasp, time_sense = recon_bart(times, kdata_sorted, ktraj_sorted, bart_files_folder, dataset_name)
    csm_gt, hollow_mask_gt, recon_grasp_gt, recon_sense_gt, time_grasp_gt, time_sense_gt = recon_bart(times_gt, kdata_sorted_gt, ktraj_sorted_gt, bart_files_folder, dataset_name_gt)
    # --- SAVE ---
    onp.savez(path_save, Y_data=Y_data, X_data=X_data, csm=csm, hollow_mask=hollow_mask, spclim=spclim, recon_grasp=recon_grasp, recon_sense=recon_sense, time_grasp=time_grasp, time_sense=time_sense, recon_fs=recon_grasp_gt, time_recon_fs=time_grasp_gt)
    # --- SAVE gifs ---
    recon_grasp_cropped = recon_grasp[recon_grasp.shape[0]//4:3*recon_grasp.shape[0]//4,recon_grasp.shape[1]//4:3*recon_grasp.shape[1]//4,:]
    recon_sense_cropped = recon_sense[recon_sense.shape[0]//4:3*recon_sense.shape[0]//4,recon_sense.shape[1]//4:3*recon_sense.shape[1]//4,:]
    recon_grasp_gt_cropped = recon_grasp_gt[recon_grasp_gt.shape[0]//4:3*recon_grasp_gt.shape[0]//4,recon_grasp_gt.shape[1]//4:3*recon_grasp_gt.shape[1]//4,:]
    save_frames_as_gif_with_pillow(dataset_recon_folder, recon_grasp_cropped / onp.max(onp.abs(recon_grasp_cropped)), filename='GRASP', vmax=1, saturation=0.8, fps=30)
    save_frames_as_gif_with_pillow(dataset_recon_folder, recon_sense_cropped / onp.max(onp.abs(recon_sense_cropped)), filename='SENSE', vmax=1, saturation=0.8, fps=30)
    save_frames_as_gif_with_pillow(dataset_recon_folder, recon_grasp_gt_cropped / onp.max(onp.abs(recon_grasp_gt_cropped)), filename='FS', vmax=1, saturation=0.8, fps=30)

n_spokes =  263
n_readout =  256
n_coils =  15
n_bins =  30
n_dims =  2
n_spokes =  2583
n_readout =  256
n_coils =  15
n_bins =  30
n_dims =  2


2025-07-20 18:15:11.972424: W external/xla/xla/service/gpu/nvptx_compiler.cc:836] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version (12.6.20). Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


The trajectories are not centered!
The trajectories are not centered!
Est. image size: 256x256x1
Done.
Done.
(256, 243, 15) (243, 256, 2)
[  1 256  12  15   1   1   1   1   1   1  30   1   1   1   1   1 ]
[256 256   1  15   1   1   1   1   1   1  30   1   1   1   1   1 ]
TV regularization: 0.010000
Regularization terms: 1, Supporting variables: 0
ADMM
Total Time: 86.280466
[  1 256  12  15   1   1   1   1   1   1  30   1   1   1   1   1 ]
[256 256   1  15   1   1   1   1   1   1  30   1   1   1   1   1 ]
Regularization terms: 0, Supporting variables: 0
conjugate gradients
Total Time: 21.375372
Est. image size: 256x256x1
Done.
Done.
(256, 2563, 15) (2563, 256, 2)
[  1 256  95  15   1   1   1   1   1   1  30   1   1   1   1   1 ]
[256 256   1  15   1   1   1   1   1   1  30   1   1   1   1   1 ]
TV regularization: 0.010000
Regularization terms: 1, Supporting variables: 0
ADMM
Total Time: 82.746904
[  1 256  95  15   1   1   1   1   1   1  30   1   1   1   1   1 ]
[256 256   1  15   1   1

In [8]:
# paths
dataset_name        = '4CH'
dataset_recon_folder      = recons_folder + dataset_name + '/'
path_save                 = train_data_folder + dataset_name + '.npz'
path2folder = raw_data_folder + dataset_name + '.mat'
# load data
bins, cbins, kdata, ktraj = load_acq_data(path2folder)
n_spokes, n_readout, n_coils, n_bins, n_dims = get_inf_about_data(ktraj, kdata, cbins)
# --- Preprocess data  ---
bins_filtered, kdata_filtered, ktraj_filtered, hb_filtered = filtering_spokes(eliminated_spokes, bins, kdata, ktraj)
# sort data
kdata_sorted, ktraj_sorted, bins_sorted, hb_sorted = sort_data_by_bin(bins_filtered, kdata_filtered, ktraj_filtered, hb_filtered)
# times
times = onp.array(bins_sorted)/n_bins 
# --- D-DIP dataset ---
Y_data, X_data, spclim = preprocess_data(times, kdata_sorted, ktraj_sorted)
# --- BART: COILS + RECONS ---
csm, hollow_mask, recon_grasp, recon_sense, time_grasp, time_sense = recon_bart(times, kdata_sorted, ktraj_sorted, bart_files_folder, dataset_name)
# --- SAVE ---
onp.savez(path_save, Y_data=Y_data, X_data=X_data, csm=csm, hollow_mask=hollow_mask, spclim=spclim, recon_grasp=recon_grasp, recon_sense=recon_sense, time_grasp=time_grasp, time_sense=time_sense)
# --- SAVE gifs ---
recon_grasp_cropped = recon_grasp[recon_grasp.shape[0]//4:3*recon_grasp.shape[0]//4,recon_grasp.shape[1]//4:3*recon_grasp.shape[1]//4,:]
recon_sense_cropped = recon_sense[recon_sense.shape[0]//4:3*recon_sense.shape[0]//4,recon_sense.shape[1]//4:3*recon_sense.shape[1]//4,:]
save_frames_as_gif_with_pillow(dataset_recon_folder, recon_grasp_cropped / onp.max(onp.abs(recon_grasp_cropped)), filename='GRASP', vmax=1, saturation=0.8, fps=30)
save_frames_as_gif_with_pillow(dataset_recon_folder, recon_sense_cropped / onp.max(onp.abs(recon_sense_cropped)), filename='SENSE', vmax=1, saturation=0.8, fps=30)

n_spokes =  2603
n_readout =  256
n_coils =  12
n_bins =  30
n_dims =  2
The trajectories are not centered!
Est. image size: 256x256x1
Done.
Done.
(256, 2583, 12) (2583, 256, 2)
[  1 256  95  12   1   1   1   1   1   1  30   1   1   1   1   1 ]
[256 256   1  12   1   1   1   1   1   1  30   1   1   1   1   1 ]
TV regularization: 0.010000
Regularization terms: 1, Supporting variables: 0
ADMM
Total Time: 70.526303
[  1 256  95  12   1   1   1   1   1   1  30   1   1   1   1   1 ]
[256 256   1  12   1   1   1   1   1   1  30   1   1   1   1   1 ]
Regularization terms: 0, Supporting variables: 0
conjugate gradients
Total Time: 19.296794


In [9]:
# paths
dataset_name        = '2CH'
dataset_recon_folder      = recons_folder + dataset_name + '/'
path_save                 = train_data_folder + dataset_name + '.npz'
path2folder = raw_data_folder + dataset_name + '.mat'
# load data
bins, cbins, kdata, ktraj = load_acq_data(path2folder)
n_spokes, n_readout, n_coils, n_bins, n_dims = get_inf_about_data(ktraj, kdata, cbins)
# --- Preprocess data  ---
bins_filtered, kdata_filtered, ktraj_filtered, hb_filtered = filtering_spokes(eliminated_spokes, bins, kdata, ktraj)
# sort data
kdata_sorted, ktraj_sorted, bins_sorted, hb_sorted = sort_data_by_bin(bins_filtered, kdata_filtered, ktraj_filtered, hb_filtered)
# times
times = onp.array(bins_sorted)/n_bins 
# --- D-DIP dataset ---
Y_data, X_data, spclim = preprocess_data(times, kdata_sorted, ktraj_sorted)
# --- BART: COILS + RECONS ---
csm, hollow_mask, recon_grasp, recon_sense, time_grasp, time_sense = recon_bart(times, kdata_sorted, ktraj_sorted, bart_files_folder, dataset_name)
# --- SAVE ---
onp.savez(path_save, Y_data=Y_data, X_data=X_data, csm=csm, hollow_mask=hollow_mask, spclim=spclim, recon_grasp=recon_grasp, recon_sense=recon_sense, time_grasp=time_grasp, time_sense=time_sense)
# --- SAVE gifs ---
recon_grasp_cropped = recon_grasp[recon_grasp.shape[0]//4:3*recon_grasp.shape[0]//4,recon_grasp.shape[1]//4:3*recon_grasp.shape[1]//4,:]
recon_sense_cropped = recon_sense[recon_sense.shape[0]//4:3*recon_sense.shape[0]//4,recon_sense.shape[1]//4:3*recon_sense.shape[1]//4,:]
save_frames_as_gif_with_pillow(dataset_recon_folder, recon_grasp_cropped / onp.max(onp.abs(recon_grasp_cropped)), filename='GRASP', vmax=1, saturation=0.8, fps=30)
save_frames_as_gif_with_pillow(dataset_recon_folder, recon_sense_cropped / onp.max(onp.abs(recon_sense_cropped)), filename='SENSE', vmax=1, saturation=0.8, fps=30)

n_spokes =  2535
n_readout =  256
n_coils =  12
n_bins =  30
n_dims =  2
The trajectories are not centered!
Est. image size: 256x256x1
Done.
Done.
(256, 2515, 12) (2515, 256, 2)
[  1 256  94  12   1   1   1   1   1   1  30   1   1   1   1   1 ]
[256 256   1  12   1   1   1   1   1   1  30   1   1   1   1   1 ]
TV regularization: 0.010000
Regularization terms: 1, Supporting variables: 0
ADMM
Total Time: 70.108620
[  1 256  94  12   1   1   1   1   1   1  30   1   1   1   1   1 ]
[256 256   1  12   1   1   1   1   1   1  30   1   1   1   1   1 ]
Regularization terms: 0, Supporting variables: 0
conjugate gradients
Total Time: 18.778538
